**In may Q & A applications, we want to allow user to have back and forth conversation like e.g.**


 ---> Who won the cricket worldcup 2019?

 ---> Who was the captain of the team?


**So we need a memory element. We neeed memory of past Q&A and some logic of incorporating those into its current thinking.**

**Step # 01: Install All the Required Packages**

In [1]:
!pip install --upgrade --quiet langchain langchain-community langchainhub langchain-chroma bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s et

In [3]:
!pip install -qU langchain-groq

In [4]:
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [5]:
import warnings
warnings.filterwarnings("ignore")

**Step # 02: Using Language Model**

In [6]:
import os
import getpass

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

llm = ChatGroq(model = "llama-3.1-70b-versatile")

··········


**Step # 03: Create a Q&A Application using Built-in Chain Constructors, create_stuff_documents_chain and create_retrieval_chain. So the basic ingredients to our solution are.**

1. retriver

2. prompt

3. LLM

In [8]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

#Indexing: Load
loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_ = ("post-content", "post-title", "post-header")
        )
    )
)

docs = loader.load()

print(f"The loaded document contains {len(docs[0].page_content)} characters")

print(f"The first 500 characters of the document: \n {docs[0].page_content[:500]}\n")


#Indexing: Split
textSplitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)

split = textSplitter.split_documents(docs)
print(f"Total text chunks are {len(split)} \n\n")
print(f"The size of the first text chunk is {len(split[0]. page_content)}\n")


#Indexing: Store
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device":"cuda"}
embeddings = HuggingFaceEmbeddings(model_name = model_name, model_kwargs = model_kwargs)

vectorStore = Chroma.from_documents(documents = split, embedding = embeddings)

#Retrieval and Generation: Retrieve
retriever = vectorStore.as_retriever(search_type = "similarity", search_kwargs = {"k": 6})
retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")
#print("Retrieved Docs \n", retrieved_docs)
retrieved_docs

#Incorporate the retriever into a question-answering chain
system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the"
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

question_answering_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(retriever, question_answering_chain)

response = rag_chain.invoke({"input":"What is task decomposition?"})
response["answer"]

The loaded document contains 43131 characters
The first 500 characters of the document: 
 

      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In

Total text chunks are 66 


The size of the first text chunk is 969



'Task decomposition is the process of breaking down a problem into smaller, more manageable tasks or subgoals. In the context of Tree of Thoughts, it can be done in three ways: (1) by using a large language model (LLM) with simple prompting, (2) by using task-specific instructions, or (3) with human inputs.'

  **Adding Chat History**

Before we go ahead consider this exchange.

Human: "What is Task Decomposition"

AI: "Task Decomposition involves breaking down complex tasks into smaller......"

Human: "What are common ways of doing it"

--> In order to answer the second question, our system need to understand that "it" refers to task decomposition.

We need to update two things about our existing application.

1. **Prompt**: Update our prompt to support historical messages as an input.

2. **Contextualizing Questions**: Add a sub-chain that takes the latest user question and reformulates it in the context of chat history. This can be though of simply as building a new "history aware" retriever.


Before we had:

query --> retriever

Now we will have:

(query, conversation history) --> LLM --> rephrased query --> retriver

**Contextualizing the Question**

In [9]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question"
    "which might reference context in the chat history,"
    "formulate a standalone question which can be understood"
    "without the chat history. Donot answer the question"
    "just reformulate it if needed and otherwise return it as is"
)


context_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, context_q_prompt
)

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the"
    "answer concise."
    "\n\n"
    "{context}"
)

qa_prompt =  ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)


rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [10]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content = ai_msg_1["answer"])
])

second_question = "What are common ways of doing it"

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

There are three common ways to perform task decomposition:

1. Using a Large Language Model (LLM) with simple prompting, such as asking for "Steps for XYZ" or "What are the subgoals for achieving XYZ?".
2. Using task-specific instructions, for example, "Write a story outline" for writing a novel.
3. With human inputs, where humans provide the task decomposition.


**Stateful Management of Chat History**

Now we have seen that how we can incorporate the "chat history" in our application logic but we are still manually updating the chat history and inserting it into each input.

In real Q&A applications, we will want some way of automatically inserting and updating chat history.

For this we can use:

BaseChatMessageHistory: Store chat history

RunnableWithMessageHistory:

This is a wrapper for LCEL chain and a BaseChatMessageHistory, this handles inserting and updating chat history after each invocation

In [11]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_groq import ChatGroq

llm = ChatGroq(model = "llama-3.1-70b-versatile")


#Indexing: Load
loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_ = ("post-content", "post-title", "post-header")
        )
    )
)

docs = loader.load()

#Indexing: Split
textSplitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = textSplitter.split_documents(docs)

#Indexing: Store
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device":"cuda"}
embeddings = HuggingFaceEmbeddings(model_name = model_name, model_kwargs = model_kwargs)
vectorStore = Chroma.from_documents(documents=splits, embedding=embeddings)

#Retrieval and Generation: Retrieve
retriever = vectorStore.as_retriever(search_type = "similarity", search_kwargs = {"k": 6})
retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")

##Contextualize Question###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question"
    "which might reference context in the chat history,"
    "formulate a standalone question which can be understood"
    "without the chat history. Donot answer the question,"
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
    )
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

#Answer Question
system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know. Don't try to make an answer. Use three sentences maximum"
    "and answer should be concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)



###Statefully manage chat history
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]

conversational_retrieval_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key = "input",
    history_messages_key = "chat_history",
    output_messages_key = "answer",
)

conversational_retrieval_chain.invoke(
    {"input": "What is task decomposition"},
    config = {
        "configurable":{"session_id": "abc123"}
    }, #constructs a key "abc123" in store.
)["answer"]

'Task decomposition refers to the process of breaking down a complex problem or task into smaller, more manageable subtasks or steps.'

In [12]:
conversational_retrieval_chain.invoke(
    {"input": "What are common ways of doing it"},
    config = {
        "configurable": {"session_id": "abc123"}
    }, #construct a key "ab123" in store
)["answer"]

'Task decomposition can be done in three ways: (1) by a Large Language Model (LLM) with simple prompting, (2) by using task-specific instructions, or (3) with human inputs.'

#**Agents**

Agents leverage the reasoning capabilties of LLMs to make decisions during execution.

1. Agents generate the input to the retriever directly, without necessarily needing us to explicitly build in contextualization, as we did above.

2. Agents can execute multiple retrieval steps in service of a query.

**Retrieval Tool**

Agents can access "tools" and manage their execution. In this case, we will convert our retriever into a LangChain tool to be wielded by the agent:



In [13]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "blog_post_retriever",
    "Searches and returns excerpts from the Autonomous Agents blog post."
)

tools = [tool]


In [14]:
tool.invoke("Task decompositon")

'(3) Task execution: Expert models execute on the specific tasks and log results.\nInstruction:\n\nWith the input and the inference results, the AI assistant needs to describe the process and results. The previous stages can be formed as - User Input: {{ User Input }}, Task Planning: {{ Tasks }}, Model Selection: {{ Model Assignment }}, Task Execution: {{ Predictions }}. You must first answer the user\'s request in a straightforward manner. Then describe the task process and show your analysis and model inference results to the user in the first person. If inference results contain a file path, must tell the user the complete file path.\n\n(3) Task execution: Expert models execute on the specific tasks and log results.\nInstruction:\n\nWith the input and the inference results, the AI assistant needs to describe the process and results. The previous stages can be formed as - User Input: {{ User Input }}, Task Planning: {{ Tasks }}, Model Selection: {{ Model Assignment }}, Task Execution

**Agent Constructor**

Now that we have defined the tools and the LLM, we can create the agent.

We will be using LangGraph to construct the agent.


In [15]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 3.3 MB/s eta 0:00:00


In [16]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(llm, tools)

In [17]:
query = "What is Task Decomposition?"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content = query)]},
):
  print(s)
  print("----")

{'agent': {'messages': [AIMessage(content='Task decomposition is a process of breaking down a complex task or problem into smaller, more manageable sub-tasks or sub-problems. This technique is used to simplify the task, identify the key components, and create a clear plan of action to accomplish the task.\n\nTask decomposition involves identifying the main goal or objective of the task, and then breaking it down into smaller tasks that are necessary to achieve that goal. Each sub-task is then further broken down into even smaller tasks, until the tasks are simple enough to be easily accomplished.\n\nTask decomposition is a useful technique for several reasons:\n\n1. Simplifies complex tasks: By breaking down a complex task into smaller tasks, it becomes easier to understand and manage.\n2. Identifies key components: Task decomposition helps to identify the key components of the task, which can help to focus efforts and resources on the most important aspects of the task.\n3. Creates a 

**Important: LangGraph comes with built in persistence, so we don't need to use ChatMessageHistory!**


**Rather, we can pass in a checkpointer to our LangGraph agent directly.**

In [18]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

So we have constructed a Conversational RAG Agent.


If we input a query that doesnot require a retrieval step, the agent doesnot execute one

In [19]:
config = {"configurable": {"thread_id": "abc123"}}

for s in agent_executor.stream(
    {"messages": [HumanMessage(content = "Hi! I am Moin")]}, config = config
):
  print(s)
  print("---")

{'agent': {'messages': [AIMessage(content='Hello Moin! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 926, 'total_tokens': 938, 'completion_time': 0.048, 'prompt_time': 0.227976451, 'queue_time': None, 'total_time': 0.275976451}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b3ae7e594e', 'finish_reason': 'stop', 'logprobs': None}, id='run-a716290b-1a6c-4144-84f0-aa1b34099fdc-0', usage_metadata={'input_tokens': 926, 'output_tokens': 12, 'total_tokens': 938})]}}
---


Further, if we input a query that does require a retrieval step, the agent generates the input to the tool:



In [22]:
#config = {"configurable": {"thread_id": "abc123"}}

for s in agent_executor.stream(
    {"messages": [HumanMessage(content = "What is Task Decomposition?")]}, config = config
):
  print(s)
  print("---")

{'agent': {'messages': [AIMessage(content='Task decomposition is the process of breaking down a complex task or problem into smaller, more manageable sub-tasks or sub-problems. This helps to identify the individual components of the task, making it easier to understand, analyze, and solve.', response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 2206, 'total_tokens': 2255, 'completion_time': 0.196, 'prompt_time': 0.562188956, 'queue_time': None, 'total_time': 0.7581889559999999}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'stop', 'logprobs': None}, id='run-deb873a7-5095-41d3-b127-4445f5b64fcb-0', usage_metadata={'input_tokens': 2206, 'output_tokens': 49, 'total_tokens': 2255})]}}
---


In [21]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content = "What according to the blog past are common ways of doing it? redo the search")]}, config = config
):
  print(s)
  print("---")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9vtk', 'function': {'arguments': '{"query":"common ways of task decomposition"}', 'name': 'blog_post_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 1177, 'total_tokens': 1250, 'completion_time': 0.292, 'prompt_time': 0.249880747, 'queue_time': None, 'total_time': 0.541880747}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_5c5d1b5cfb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1ed92373-4c7a-4886-a421-3670a782827c-0', tool_calls=[{'name': 'blog_post_retriever', 'args': {'query': 'common ways of task decomposition'}, 'id': 'call_9vtk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1177, 'output_tokens': 73, 'total_tokens': 1250})]}}
---
{'tools': {'messages': [ToolMessage(content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. I